# Powerfolio!

<img src="images/logo.png" width="150" title="stock_flex" align="center">

---

### Import Dependancies
Import system, data analytics, finance API, database tools, and visualization libraries

In [2]:
# System
import os
import time, sys
from pathlib import Path
from dotenv import load_dotenv
import requests
from datetime import date, datetime, timedelta
from ast import literal_eval as make_tuple

# Data analytics
import pandas as pd
from pandas import DataFrame, MultiIndex
from typing import List, NewType
import numpy as np

# Database tools
import sqlite3

# Visualization
import panel as pn
import panel.widgets as pnw
import plotly.express as px
import plotly.graph_objects as go
pn.extension('plotly')
pn.extension()
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt

# Finance
import alpaca_trade_api as tradeapi
import quandl as ql
import finnhub

import warnings
warnings.filterwarnings('ignore')

# Local dependancies
import MyPortfolioSimulator as mp

ModuleNotFoundError: No module named 'finnhub'

---

## Data Connections
- Static Data Connections
- Dynamic Data Connections

### Static Data Connections

##### Stock Ticker Lists

In [ ]:
# Get tickers within S&P500 index
sp500_tickers_path = Path('resources/sp500_tickers.csv')
sp500_tickers = pd.read_csv(sp500_tickers_path).sort_values(by="Symbol")

In [ ]:
# S&P 500 ticker list test
sp500_tickers.head(10)

In [ ]:
# Get fundamental data from csv
stock_fundamentals_data_path = Path("resources/fundamental_data.csv")
stock_fundamentals_df = pd.read_csv(stock_fundamentals_data_path)

In [ ]:
# Fundamental values test
stock_fundamentals_df.head()

### Dynamic Data Connections

#### Stock Price Data

In [ ]:
# Alpaca API connector
load_dotenv("../resources/api_keys.env")

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
alpaca_api_key,
alpaca_secret_key,
api_version = "v2"
)

In [ ]:
# Test Alpaca Trade API key
type(alpaca_api_key)

In [ ]:
# Test Alpaca Trade API secret key
type(alpaca_secret_key)

#### Fundamental Stock Data

In [ ]:
# FinnHub API connector
load_dotenv("../resources/api_keys.env")

# Set FinnBug API key
finnhub_api_key = os.getenv("FINNHUB_API_KEY")

# Create FinnHub API object
finnhub_client = finnhub.Client(api_key=finnhub_api_key)

# Method to obtain json data from FinnHub
def finnhub_data(ticker):
    
    data = finnhub_client.company_basic_financials(ticker, "")
    data_df = pd.DataFrame(data)
    time.sleep(1)
    
    return data_df

type(finnhub_api_key)

#### Bond Data

In [ ]:
# Treasury bonds
def treasury_data():
    return ql.get("USTREASURY/YIELD")

In [ ]:
# Quandl treasure data test
treasury_data().head()

---

## Data Parsing

##### Stock Data Parsing Methods
A collection of methods for parsing S&P 500 tickers, calling Alpaca Trade API, and querying stock_prices.db database

#### Alpaca API Methods

In [ ]:
# Get prices for tickers withing a given index or sector
def stock_prices_from_api(ticker_list, start_date, end_date):
    '''Returns pd.DataFrame with prices for the given tickers
    
    ...
    
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
    start_date : str() - string with date in following format YYYY-MM-DD
    end_date: str() - string with date in following format YYYY-MM-DD 
    
    
    Returns
    -------
    result_df : pd.DataFrame with securities price data
    '''
   
    # Get list of tickers from the tickers_df list or tickers_df DataFrame 
    tickers = ticker_list
    
    # Parse start and end dates
    start_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()
    
    # Connect to Alpaca API and get data
    """Condition handling: 
        a. Alpaca API 422 Client Error if more than 100 tickers are passed - COMPLETE
        b. Alpaca API data max row limit of 1000 - PENDING"""
    
    
    # a. Alpaca API condition handling, sending 100 tickers at a time
    # Declate a pd.DataFrame
    result_df = pd.DataFrame()
    tickers_n = 50
    
    for i in range(0, len(tickers), tickers_n):
        # Slice the ticker list into lists of 50 tickers
        sliced_tickers = tickers[i:i + tickers_n] 
        
        temp_df = api.get_barset(
        sliced_tickers,
        timeframe = "1D",
        start = start_date,
        end = end_date,
        limit = 1000).df

        # Append temporary dataframe to result_df
        result_df = pd.concat([result_df, temp_df], axis = "columns", join = "outer")
        time.sleep(0.1)
        
    return result_df

In [ ]:
# Alpaca Trade API stock data test
stock_prices = stock_prices_from_api(list(sp500_tickers.head(10)["Symbol"]), "2020-05-01", '2021-05-13')
stock_prices.head()

In [ ]:
# Multiindex slicing
idx = pd.IndexSlice
index = list(stock_prices.transpose().index.get_level_values(0))
formatted_stock_prices = stock_prices.xs('close', axis=1, level=1, drop_level=False).droplevel(1, axis=1)
formatted_stock_prices.head()

#### Local <code>stock_price.db</code> Database Methods

In [ ]:
# Rebuilds database for Alpaca API stock prices if necessary
def create_stock_prices_db():
    '''This method creates a SQLite database that will store all of ticker daily prices;
    This method is created to resolve the Alpaca API request limits that can greatly slow down large data requests
    '''
    
    print("This will restore the 'stock_prices' database and delete all of stored values, are you sure? y/[n]")

    conn = sqlite3.connect('resources/stock_prices.db')
    
    
# Seed stock price database
def seed_stock_prices_database(tickers_df):
    '''Seeds stock_prices database that houses all prices for tickers within an index or sector'''

    # Get data for dates indicated
    data = stock_prices_from_api(tickers_df, "2015-01-01", "2018-01-01")
    
    # Transpose dataframe
    data = data.transpose()
    
    # Connect to databse
    conn = sqlite3.connect('resources/stock_prices.db') 
    c = conn.cursor()
    
    # Iterate over dataframe and populate the database
    for ticker, sub_frame in data.groupby(level=0):
        
        ticker = "_" + ticker.replace('.','_')
        
        list_of_tables = c.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{ticker}';").fetchall()
        if list_of_tables == []:
            sub_frame.transpose().to_sql(ticker, conn, if_exists='append')
        else:
            sub_frame.transpose().to_sql(ticker, conn, if_exists='append')
    
    conn.commit()
    conn.close()

    

# Update stock price database     
def update_stock_prices_database(tickers_df):
    '''Updates stock_prices database that houses all prices for tickers within an index or sector'''
    
    # Get today's date
    today_date = date.today().strftime("%Y-%m-%d")
    
    # Connect to databse
    conn = sqlite3.connect('resources/stock_prices.db') 
    c = conn.cursor()
    
    # Iterate over dataframe and populate the database
    ticker_count = 0
    for index, ticker_data in tickers_df.iterrows():
        
        ticker = "_" + ticker_data["Symbol"].replace('.','_')
        
        # Get list of all tables in databse
        list_of_tables = c.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{ticker}';").fetchall()
        
        # If table does not exist, create and seed
        if list_of_tables == []: # If tables does not exist
            # Get data from Alpaca API
            data = stock_prices_from_api(ticker_data, "2015-01-01", "2018-01-01")
            
            # Write stock data to databse
            data.to_sql(ticker, conn, if_exists='append')
            
        else: # If table does exist
            try:
                # Get last date from the table
                last_date = c.execute(f"SELECT time FROM {ticker} ORDER BY time DESC LIMIT 1").fetchall()
                last_date = ''.join(last_date[0]).split(" ")[0]

                # Get next date
                next_date = datetime.strptime(last_date, '%Y-%m-%d') + timedelta(days=1)

                # Change date format for Alpaca
                next_date = next_date.strftime("%Y-%m-%d")
                
                # Get today's date
                today_date = datetime.today().strftime("%Y-%m-%d")
                
                if last_date < today_date:

                    # Get data from Alpaca API
                    data = stock_prices_from_api(ticker_data, last_date, today_date)

                    # Write data to database
                    data.to_sql(ticker, conn, if_exists='append')
                
                else:
                    pass
            except:
                pass
            
        ticker_count += 1
    print(f"Processing {ticker} | {ticker_count}/{len(tickers_df.index)}")
    
    conn.commit()
    conn.close()
    
    
    
# Get stock prices from database
def stock_prices_from_db(ticker_df, start_date, end_date):
    '''Returns a pd.DataFrame with stock price data for tickers passed in ticker_df and 
    filtered by dates passed in start_date and end_date
    
    ...
    
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
    start_date : str() - string with date in following format YYYY-MM-DD
    end_date: str() - string with date in following format YYYY-MM-DD 
    
    
    Returns
    -------
    result_df : pd.DataFrame with securities price data
    '''
    # Connect to database
    conn = sqlite3.connect('resources/stock_prices.db') 
    c = conn.cursor()
    
    # Cast dates to pandas datetime
    start_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()
    
    # Declare result_df
    result_df = pd.DataFrame()
    
    # Iterate over tickers
    for ticker in list(ticker_df["Symbol"]):
        
        try:
            # Parse ticker name
            _ticker = "_" + ticker.replace('.','_')

            # Query databse
            data = pd.read_sql_query(f"SELECT * FROM {_ticker}", conn)

            # Filter database by dates
            data = data[(data['time'] >= start_date) & (data['time'] <= end_date)].set_index(['time'])

            # Concatenate dataframes
            result_df = pd.concat([result_df, data], axis = "columns", join = "outer")
        
        except:
            print(f"Ticker {ticker} not found in database.")
        
    conn.commit()
    conn.close()
        
    # Create a multi-index 
    ## Transpose the df
    result_df = result_df.transpose()
    
    ## Get dataframe keys
    keys = []
    for key in result_df.index:
        keys.append(make_tuple(key))

    ## Build an index
    index = pd.MultiIndex.from_tuples(keys, names=('Symbol', 'Data'))

    ## Set index
    result_df = result_df.set_index(index).transpose()
        
    return result_df


# Get database date-range
def get_db_daterange():
    
    # Connect to database
    conn = sqlite3.connect('resources/stock_prices.db') 
    c = conn.cursor()
    
    last_date = pd.read_sql_query("SELECT time FROM _MSFT ORDER BY time DESC LIMIT 1", conn)['time'][0].split(" ")[0]
    
    first_date = pd.read_sql_query("SELECT time FROM _MSFT ORDER BY time ASC LIMIT 1", conn)['time'][0].split(" ")[0]
    
    
    conn.commit()
    conn.close()
    
    return (first_date, last_date)

def offline_application_set_up():
    '''Helper method - sets up offline access to Dashboard'''
    create_stock_prices_db()
    seed_stock_prices_database(sp500_tickers)
    update_stock_prices_database(sp500_tickers)

---

## Technical Methods

### Technical Data Methods

In [ ]:
# RSI method : Calculate the RSI indicator
def rsi_table (df, days):
    '''Returns a pd.DataFrame with Relative Strength Index (RSI) column appended
        RSI formula = 100 – (100 / (1 + RS) or can also use 100 * up / (Up + Down)
        Where RS (relative strengh)  = Up / Down
        Where Up = rolling average price up over the time window obeserved
        Where Down = rolling average price down over the time window obeserved
    
        Parameters
        ----------
        df : pd.DataFrame - dataframe to be processed
        days : int() - numbers of days for RSI calcualtion
    
        Returns
        -------
        result_df : pd.DataFrame - dataframe with RSI column appended, calcualted daily for 
        timeperiod specified by days
    '''
    
    # Filter datafrance to clumn "close" only to facilitate the calculs
    stocks_close = df.iloc[:,df.columns.get_level_values(1)=='close']
    # Swap the column multilevel index to facilitate the calculs
    stocks_close = stocks_close.swaplevel(0,1,axis=1)
    
    # Calculate the movement on the price compared to the previous day closing price
    movement_1d = stocks_close - stocks_close.shift(1)
    movement_1d.rename(columns={'close':'movement_1d'}, level=0, inplace=True)

    # Define a sub-function to calculate the RSI
    def rsi (price):
        up = price[price>0].mean()
        down = abs(price[price<0]).mean()
        return 100 * up / (up + down)
        
    # Calculate the RSI and add it to a dataframe
    rsi_df = movement_1d.rolling(window=days).apply(rsi)
    rsi_df.rename(columns={'movement_1d':'RSI'}, level=0, inplace=True)
    
    # Remove the first dates that return NaN from the rolling days calculation
    rsi_df = rsi_df[days:]

    return rsi_df

In [ ]:
# rsi_table test
days = 14
rsi_df = rsi_table (stock_prices, days)
rsi_df.to_csv("resources/rsi.csv")
rsi_df.head()

In [ ]:
# MACD method : Calculate the MACD indicator with its Signal line
def macd_table(df, short_window, long_window, signal_window):
    ''' Moving Average Convergence Divergence (MACD) 
        Returns a pd.DataFrame with MACD and its Signal line column appended
        MACD formula = (12-day EMA - 26-day EMA)
        Signal line = MACD 9-day EMA
    
        Parameters
        ----------
        df : pd.DataFrame - dataframe to be processed
        short_ema : int() - short-term EMA for MACD calculation => default should be 12 days
        long_ema : int() - long-term EMA for MACD calculation => defaultshould be 26 days

        Returns
        -------
        result_df : pd.DataFrame - dataframe with MACD and Signal line column appended, calcualted daily for 
        timeperiod specified by days
    '''

    # Filter datafrance to clumn "close" only to facilitate the calculs
    stocks_close = df.iloc[:,df.columns.get_level_values(1)=='close']
    stocks_close = stocks_close.swaplevel(0,1,axis=1)
    
    # Define a sub-function to calculate the MACD
    def macd (price):
        short_ema = price.ewm(span=short_window, adjust=False).mean()
        long_ema = price.ewm(span=long_window, adjust=False).mean()
        macd_value = short_ema - long_ema
        return macd_value
    
    # Calculate the MACD and add it to a dataframe
    macd = stocks_close.apply(macd)
    macd.rename(columns={'close':'MACD'}, level=0, inplace=True)

    # Calculate the Signal line value
    signal = macd.ewm(span=9, adjust=False).mean()
    signal.rename(columns={'MACD':'MACD_Signal'}, level=0, inplace=True)
    
    # Concatenate/append both indicators to a new dataframe
    result_df = pd.concat([macd, signal], axis=1, join='inner')

    return result_df

In [ ]:
# MACD method test
short_window = 12
long_window = 26
signal_window = 9
combined_macd = macd_table(stock_prices, short_window, long_window, signal_window)
combined_macd.to_csv("resources/macd.csv")
combined_macd.head()

In [ ]:
# Bollinger Bands method : 
def bbands_table(df, length, numstd):
    '''Bollinger Bands (BB)
    returns average, upper band, and lower band
    
    Parameters
    ----------
    df : pd.DataFrame - dataframe to be processed
    lenght : int() - numer of prices we want to use to observe the average price 
    numstd : int() - number of Standard deviation we want to use to calculate the bands
    
    Returns
    -------
    result_df : pd.DataFrame - dataframe with RSI column appended, calcualted daily for 
    timeperiod specified by days
    '''
    
    # Filter datafrance to clumn "close" only to facilitate the calculs
    stocks_close = df.iloc[:,df.columns.get_level_values(1)=='close']
    stocks_close = stocks_close.swaplevel(0,1,axis=1)


    def bb_upband(price):
        #avg = pd.stats.moments.rolling_mean(price,length)
        avg = price.rolling(window= length).mean()
        #std = pd.stats.moments.rolling_std(price,length)
        std = price.rolling(window= length).std()

        upband = avg + (std*numstd)
        return np.round(upband,3)

    def bb_dnband(price):
        #avg = pd.stats.moments.rolling_mean(price,length)
        avg = price.rolling(window= length).mean()
        #std = pd.stats.moments.rolling_std(price,length)
        std = price.rolling(window= length).std()

        dnband = avg - (std*numstd)
        return np.round(dnband,3)

    bb_avg_df = stocks_close.rolling(window= length).mean()
    bb_avg_df.rename(columns={'close':'BB_Avg'}, level=0, inplace=True)
    
    bb_upband_df = stocks_close.apply(bb_upband)
    bb_upband_df.rename(columns={'close':'BB_Upband'}, level=0, inplace=True)

    bb_dnband_df = stocks_close.apply(bb_upband)
    bb_dnband_df.rename(columns={'close':'BB_Downband'}, level=0, inplace=True)

    # Concatenate/append both values to the original dataframe
    result_df = pd.concat([bb_avg_df, bb_dnband_df, bb_upband_df], axis=1, join='inner')

    return result_df

In [ ]:
# Bollinger Band method test
length = 30
numstd = 2
combined_bb = bbands_table(stock_prices, length, numstd)
combined_bb.to_csv("resources/bb.csv")
combined_bb.tail()

In [ ]:
def rsi_trader(tickers_list : list(), 
               lower_level : int(), 
               upper_level : int(), 
               start_date : str(), 
               end_date : str(), 
               rsi_range : int()):
    '''Returns a pd.DataFrame of a portoflio where position entry and exit are 
        determined by Relative Strength Index (RSI) values provided by the user.
    
        Parameters
        ----------
        tickers_list : list() - list of tickers to be traded
        lower_level : int() - lower level of RSI provided by user
        upper_level : int() - upper level of RSI provided by user
        start_date : str() - analysis start date string "YYYY-MM-DD"
        end_date : str() - analysis end date string "YYYY-MM-DD"
        rsi_range : int() - numbers of days for RSI calcualtion
    
        Returns
        -------
        result_df : pd.DataFrame - RSI traded portfolio
    '''
    
    # Get stock prices form Alpaca Trade API
    stock_prices = stock_prices_from_api(tickers_list, start_date, end_date)
    
    # Collapse multi-index to yield a single-index df with 'close' prices
    try:
        formatted_stock_prices = stock_prices.xs('close', axis=1, level=1, drop_level=False).droplevel(1, axis=1)
    except:
        formatted_stock_prices = pd.DataFrame({'time' : [datetime.strptime('2020-01-01', "%Y-%m-%d")], 'close' : [0.0]})
        print("RSI Portfolio Builder received an empty pd.DataFrame of stock prices")
        raise
        
    # Declare RSI df
    rsi_df = pd.DataFrame()   
    
    # Declare arbitrary cash position df and seed it
    cash_seed = 1000000.0
    cash_position = {}
    cash_position[0] = {'time' : formatted_stock_prices.index[0], 'close' : cash_seed}
    
    # Track long stock positions
    long_stock_position_tracker = {}
    for ticker in tickers_list:
        long_stock_position_tracker[ticker] = False
    
    # Build RSI trader positions over time
    ## iterate ticker colums
    for ticker in tickers_list:
        
        ### Declate dict for stock positions
        stock_seed = 0.0
        stock_position = {}
        stock_position[0] = {'time' : formatted_stock_prices.index[0], 'close' : stock_seed, 'rsi' : 0, 'stock_price' : 0, 'cash' : cash_seed/len(tickers_list)}
        
        ### generate df for ticker
        ticker_df = pd.DataFrame(formatted_stock_prices[ticker]).reset_index()

        # Calculate RSI
        delta = ticker_df[ticker].diff()
        move_up = delta.clip(lower=0)
        move_down = -1*delta.clip(upper=0)
        ema_up = move_up.ewm(com=rsi_range, adjust=False).mean()
        ema_down = move_down.ewm(com=rsi_range, adjust=False).mean()
        relative_strenght = ema_up/ema_down
        
        # add RSI to df
        ticker_df['rsi'] = 100 - (100/(1 + relative_strenght))
     
        # Skip first 14 days
        ticker_df = ticker_df.iloc[14:]
           
        #### iterate over the trading days
        for day in range(len(ticker_df.index)):
            ### get data for previous and current days and get RSI
            date = ticker_df.iloc[day,:]['time']
            stock_price = ticker_df.iloc[day,:][ticker]
            previous_day_rsi  = float(ticker_df.iloc[day-1, 2])
            current_day_rsi = float(ticker_df.iloc[day, 2])

            #### Trade logic - Open Long Position
            if previous_day_rsi < float(lower_level) and current_day_rsi > float(lower_level):

                ##### Check if long stock -> pass
                if long_stock_position_tracker[ticker] == True:
                    ## Adjust position
                    stock_position[day] = {'time' : date, 'close' : stock_position[day-1]['close'], 'rsi' : current_day_rsi,  'stock_price' : stock_price, 'cash' : stock_position[day-1]['cash']}


                #### Check if not long stock -> Go long
                elif long_stock_position_tracker[ticker] == False:
                    # Long position entry triggered

                    ## Adjust position
                    stock_adj = (cash_seed/len(tickers_list))/stock_price
                    cash_adj = 0

                    stock_position[day] = {'time' : date, 'close' : stock_adj, 'rsi' : current_day_rsi,  'stock_price'  : stock_price, 'cash' : cash_adj}

                    long_stock_position_tracker[ticker] = True

            ### Trade Logic - Close Long Position
            elif previous_day_rsi > float(upper_level) and current_day_rsi < float(upper_level):

                #### Check if not long stock -> pass
                if long_stock_position_tracker[ticker] == False:
                    ## Adjust position
                    try:
                        stock_position[day] = {'time' : date, 'close' : stock_position[day-1]['close'], 'rsi' : current_day_rsi,  'stock_price' : stock_price, 'cash' : stock_position[day-1]['cash']}
                    except:
                        stock_position[day] = {'time' : date, 'close' : 0, 'rsi' : current_day_rsi,  'stock_price' : stock_price, 'cash' : cash_seed/len(tickers_list)}

                #### Check if long stock -> Close long
                if long_stock_position_tracker[ticker] == True:
                    # Long position exit triggered
                    ## Adjust cash position
                    cash_adj = stock_position[day-1]['cash'] + stock_position[day-1]['close'] * stock_price
                    stock_adj = 0

                    stock_position[day] = {'time' : date, 'close' : stock_adj, 'rsi' : current_day_rsi,  'stock_price' : stock_price, 'cash' : cash_adj}

                    long_stock_position_tracker[ticker] = False

            ### Trade Logic - No Trade
            else:
                ## Adjust position
                try:
                    stock_position[day] = {'time' : date, 'close' : stock_position[day-1]['close'], 'rsi' : current_day_rsi,  'stock_price' : stock_price, "cash" : stock_position[day-1]['cash']}
                except:
                    stock_position[day] = {'time' : date, 'close' : 0, 'rsi' : current_day_rsi,  'stock_price'  : stock_price, 'cash' : cash_seed/len(tickers_list)}
                
        # Transpose stock_position_df   
        stock_position_df = pd.DataFrame(stock_position).transpose()
        
        # Set index to tim
        stock_position_df = stock_position_df.set_index('time')
        
        # Calculate stock_position_df positon value
        stock_position_df[ticker] = (stock_position_df['close'] * stock_position_df['stock_price']) + stock_position_df['cash']
        
        # Concatentate stock_position_df into master df
        rsi_df = pd.concat([rsi_df,stock_position_df[ticker]], axis='columns', join='outer')
    
    # Get total portfolio value
    rsi_portfolio_sum = rsi_df.sum(axis=1)/cash_seed-1

    return rsi_portfolio_sum

In [ ]:
# RSI Trader method test
tickers = ['DLR','EQIX', 'PRU']

lower_level = 30
upper_level = 70
start_date = "2020-05-01"
end_date = "2021-05-01"
rsi_range = 14
rsi_trader(tickers, lower_level, upper_level, start_date, end_date, rsi_range).head()

---

## FundamentalMethods

#### Fundamental Data Methods

In [ ]:
# Get stock fundamental data from FinnHub
def generate_stock_fundamentals_from_finnhub(tickers_df):
    '''Returns pd.DataFrame with fundamentals of tickers within tickers_df
    
    ...
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
        
    Returns
    -------
    result_df : pd.DataFrame - securities fundamental data
    '''
    
    result_df = pd.DataFrame()
    
    
    for ticker in tickers_df['Symbol']:
        fundamental_data = finnhub_data(ticker)
        result_df = pd.concat([result_df, fundamental_data], axis='rows', join="outer")
        
        
    # Parse the dataframe
    result_df = result_df.reset_index().set_index('symbol')
    result_df = result_df.drop('metricType', 1)
    result_df.columns = ['metric_type', 'metric', 'series']
    
    return result_df

In [ ]:
# Fundamental methods test
generate_stock_fundamentals_from_finnhub(sp500_tickers.head(10)).head()

In [ ]:
# Refresh stock fundamental data csv
def refresh_fundamentals_csv(tickers_df):
    '''Helper method - returns pd.DataFrame with fundamentals of tickers within tickers_df
    and/or calls to generate a refreashed dataset.
    
    Generates a CSV file with updated Fundamental values
    
    ...
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
    '''
    result_df = generate_stock_fundamentals_from_finnhub(tickers_df)
    result_df.to_csv("resources/fundamental_data.csv", mode='w')

In [ ]:
# Fundamental data filter
def fundamental_data_query(tickers_df, stock_fundamentals_df, fundamental_indicator_keys):
    '''Returns a pd.DataFrame of fundamental data filtered by user input range
    
    ...
    
    Parameters
    ----------
    tickers_df : pd.DataFrame - dataframe to be processed, contains tickers
    fundamental_indicator_key : str() - keyword for fundamental indicator requested
    
        Fundamental indicator keys ->
        
        P/E Ratio : [pe_ratio]
        EPS (Earnings per Share) : [eps]
        Annual Dividend : [dividend]
        Beta (vs. S&P 500) : [beta]
        EBIDT : [ebidt]
        Quick Ratio : [quick_ratio]
        3 Year Revenue Growth : [rev_growth]
        Free Cash Flow : [cash_flow]
    
    lower_bound : int() or float() - lower bound for fundamental value filter, default = -1000000
    upper_bound : int() or float() - upper bound for fundamental value filter, default = 1000000
     
     
     
    Returns
    -------
    result_df : pd.DataFrame - dataframe with ticker and filtered fundamental data
    '''

    fund_indicators_dict = {
        'pe_ratio' : 'peNormalizedAnnual',
        'eps' : 'epsNormalizedAnnual',
        'dividend' : 'dividendsPerShareTTM',
        'beta' : 'beta',
        'ebidt' : 'ebitdPerShareTTM',
        'quick_ratio' : 'quickRatioAnnual',
        'rev_growth' : 'revenueGrowth3Y',
        'free_cash_flow' : 'freeCashFlowAnnual'   
    }

    # Declare result_df
    result_df = pd.DataFrame()
    
    # Declate tickers list
    tickers_list = tickers_df['Symbol']
    
    # Declare fundamental data df and filter by ticker df
    data_df = stock_fundamentals_df[stock_fundamentals_df.symbol.isin(tickers_list)]
    data_df = data_df.set_index(['symbol'])
    
    # Extract requested fundamental data
    for ind in fundamental_indicator_keys:
        df = data_df[data_df['metric_type'] == fund_indicators_dict[ind]]
        result_df = pd.concat([result_df, df], axis = 1, join = 'outer')
    
        # Clean up df
        result_df = result_df.drop(columns = ['metric_type', 'series'])
        result_df = result_df.rename(columns = {'symbol' : 'ticker', 
                                'metric' : ind})
        
            # Convert all df values to numeric
        result_df[ind] = result_df[ind].apply(pd.to_numeric)

    
    
    return result_df

In [ ]:
# Fundamental data filter test
fundamental_values = fundamental_data_query(sp500_tickers, stock_fundamentals_df, ['eps', 'pe_ratio', 'dividend','ebidt','quick_ratio','rev_growth','free_cash_flow'])
fundamental_values.head()

### Fundamental Portfolio Methods

In [ ]:
# Filter stocks by Dashboard selection
def filter_stocks_by_fundamentals(pe_range: tuple, 
                                  eps_range: tuple,
                                  dividend_range: tuple):
    '''
    '''
    
    # Filter PE and EPS
    min_pe = pe_range[0]
    max_pe = pe_range[1]
    
    min_eps = eps_range[0]
    max_eps = eps_range[1]
    
    min_dividend = dividend_range[0]
    max_dividend = dividend_range[1]
    
    # Filer fundamental_values dataframe by value range
    result_df = fundamental_values[(fundamental_values['eps'] >= min_eps) & 
                                   (fundamental_values['eps'] <= max_eps) & 
                                   
                                   (fundamental_values['pe_ratio'] >= min_pe) & 
                                   (fundamental_values['pe_ratio'] <= max_pe) &
                                   
                                   (fundamental_values['dividend'] >= min_dividend) & 
                                   (fundamental_values['dividend'] <= max_dividend)
                                  ]
    
    return result_df

In [ ]:
# Filter stocks by Dashboard selection test
pe_range = (0,200)
eps_range = (0,200)
dividend_range = (4, 10)

filter_stocks_by_fundamentals(pe_range, eps_range, dividend_range).head()

In [ ]:
# List of dates between two user selected dates
def list_of_dates_between_two_dates(start_date: str,
                                    end_date: str):  
    '''
    Helper function to ger a range of dates between two dates
    '''
    return (pd.date_range(start_date, end_date-timedelta(days=1), freq='d').strftime("%Y-%m-%d").to_list())

In [ ]:
# list_of_dates_between_two_dates test
start_date = datetime.strptime("2020-01-02", '%Y-%m-%d')
end_date = datetime.strptime("2020-02-02", '%Y-%m-%d')

list_of_dates_between_two_dates(start_date, end_date)

In [ ]:
# Build fundamental portfolio
def get_fundamental_portfolio(ticker_list,
                                start_date: str = "2020-01-01", 
                                end_date: str = "2021-05-01"):
    '''Returns pd.DataFrame weighted cumulative product portfolio
    
    ...
    Parameters
    ----------
    ticker_list : list - contains tickers to be processed
    start_date : str() - analysis start date string "YYYY-MM-DD"
    end_date : str() - analysis end date string "YYYY-MM-DD"
        
    Returns
    -------
    result_df : pd.DataFrame - dataframe with ticker and filtered fundamental data
    '''
    
    # Get stock prices for given stocks 
    stock_prices = stock_prices_from_api(ticker_list, start_date, end_date)
    
    idx = pd.IndexSlice
    index = list(stock_prices.transpose().index.get_level_values(0))
    try:
        formatted_stock_prices = stock_prices.xs('close', axis=1, level=1, drop_level=False).droplevel(1, axis=1)
    except:
        formatted_stock_prices = pd.DataFrame({'time' : ['2020_01_01'], 'close' : [0.0]})
    
    # For equal weight portfolio
    ntickers = len(ticker_list)
    weights = np.full(ntickers, (1.0 / ntickers))
    
    
    equal_weighted_portfolio = formatted_stock_prices.pct_change().dot(weights).cumsum().dropna()
        
    return equal_weighted_portfolio

In [ ]:
# Build fundamental portfolio test
portfolio = pd.DataFrame(get_fundamental_portfolio(['AAPL','MSFT','TSLA'], "2020-01-01", "2021-05-01"))
portfolio = portfolio.rename(columns = {0 : 'close'})
portfolio.head()

In [ ]:
# Fundumental portfolio plot
portfolio.hvplot(x='time',
                 y='close',
                 title='Fundamental Portfolio',
                 ylabel='Portfolio Returns, %',
                 height=700,
                 legend='right')

In [ ]:
# Diplay technical analysis plot
def display_technical_chart(ticker, ohlcv, rsi, macd, start_date, end_date):
    '''Ploting chart
        Returns 3 charts: RSI, Price/volume and MACD
    
        Parameters
        ----------
        ticker : str() - symbol of teh ticker we select
        ohlcv : pd.DataFrame - initial dataframe to be processed with open / high / low / close / volume
        rsi : pd.DataFrame - dataframe to be processed with RSI
        macd : pd.DataFrame - dataframe to be processed with MACD

        start_date : str() - string with date in following format YYYY-MM-DD
        end_date : str() - string with date in following format YYYY-MM-DD
    
        Returns
        -------
        3 charts
    '''
    
    
    ############# create a new DF filtered on one ticker ###############
    
    # Swap column levels for RSI_DF and MACD_DF to make sure we have the ticker in level 0
    rsi_df= rsi.swaplevel(0,1,axis=1)
    
    
    rsi_df_sliced= rsi_df.loc[start_date:end_date,ticker]
    macd_df= macd.swaplevel(0,1,axis=1)
    macd_df_sliced= macd_df.loc[start_date:end_date,ticker]
    ohlcv_df_sliced=ohlcv.loc[start_date:end_date,ticker]
    
    # Concat the dfs
    chart_ta_df = pd.concat([ohlcv_df_sliced, rsi_df_sliced, macd_df_sliced], axis=1, join='inner')

    
    #################### Chart ######################
    
    
    # Create the figure
    fig = plt.figure(figsize=(15,10))
   
    # Create the first subplot = Price
    ax1 = plt.subplot2grid((6,4), (1,0), rowspan=4, colspan=4)
    ax1 = chart_ta_df['close'].plot(color='b', label='Price')
    ax1.grid(True, color='c', linestyle='--')
    ax1.yaxis.label.set_color("k")
    ax1.tick_params(axis='y', colors='k')
    ax1.tick_params(axis='x', colors='k')
    ax1.set_facecolor('w')
    plt.ylabel('Stock price and Volume')
   
    # Add the volume within the same subplot
    # Retrieve volumes into a list
    volume = chart_ta_df['volume']
    # Share the same x-axis
    ax1v= ax1.twinx()
    ax1v.fill_between(chart_ta_df.index, volume,0, color='darkturquoise', alpha=0.5, label='Volume')
    # To avoid having a second grid overlaping the first one
    ax1v.grid(False)
    # Set the limit for the second y-axis and make it look smaller by multiplying the max volume
    ax1v.set_ylim(0, 3*volume.max())
    ax1v.axes.yaxis.set_ticklabels([])
    ax1v.tick_params(axis='y', colors='k')
    ax1v.yaxis.label.set_color("k")
    ax1v.spines['bottom'].set_color('midnightblue')
    ax1v.spines['top'].set_color('midnightblue')
    ax1v.spines['left'].set_color('midnightblue')
    ax1v.spines['right'].set_color('midnightblue')
    ax1v.legend()

    # Create a second subplot for the RSI
    ax0 = plt.subplot2grid((6,4), (0,0), sharex=ax1, rowspan=1, colspan=4)      
    ax0 = chart_ta_df['RSI'].plot(color='r')
    ax0.axhline(70, color='m')
    ax0.axhline(30, color='m')
    ax0.set_yticks([30,70])
    ax0.yaxis.label.set_color("w")
    ax0.tick_params(axis='y', colors='k')
    ax0.tick_params(axis='x', colors='k')
    ax0.grid(True, color='c', linestyle='--')
    ax0.set_facecolor('w')
    ax0.spines['bottom'].set_color('midnightblue')
    ax0.spines['top'].set_color('midnightblue')
    ax0.spines['left'].set_color('midnightblue')
    ax0.spines['right'].set_color('midnightblue')
    plt.ylabel('RSI')

    # Create a third subplot for the MACD
    ax2= plt.subplot2grid((6,4), (5,0), sharex=ax1, rowspan=1, colspan=4)
    ax2= chart_ta_df['MACD'].plot(color='limegreen', linewidth=2, label='MACD')
    ax2.tick_params(axis='x', colors='k')
    ax2.tick_params(axis='y', colors='k')
    ax2.grid(True, color='c', linestyle='--')
    ax2.set_facecolor('w')
    ax2.spines['bottom'].set_color('midnightblue')
    ax2.spines['top'].set_color('midnightblue')
    ax2.spines['left'].set_color('midnightblue')
    ax2.spines['right'].set_color('midnightblue')
    plt.ylabel('MACD', color='k')
   
    # Add the Signal line to the same subplot
    # share the same x-axis
    ax_signal= ax2.twinx()
    ax_signal= chart_ta_df['MACD_Signal'].plot(color='orange',linestyle='--', label='Signal')
    ax_signal.spines['bottom'].set_color('midnightblue')
    ax_signal.spines['top'].set_color('midnightblue')
    ax_signal.spines['left'].set_color('midnightblue')
    ax_signal.spines['right'].set_color('midnightblue')
    plt.legend()

    return fig

In [ ]:
# display technical chart test
stock_prices_for_technical_chart = stock_prices_from_api(["TSLA"],
                                                        "2020-01-01",
                                                        "2021-01-01")
rsi_df = rsi_table(stock_prices_for_technical_chart, 14)

macd_df = macd_table(stock_prices_for_technical_chart, 12, 26, 9)

technical_chart_panel = pn.pane.Matplotlib(display_technical_chart('TSLA', 
                                                         stock_prices_for_technical_chart, 
                                                         rsi_df, 
                                                         macd_df,
                                                         "2020-01-01",
                                                         "2021-01-01"))